In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Dense

Data

In [0]:
path_to_file = "shakespeare.txt"
# We're gonna use shakespeare sonnet for data
text = open(path_to_file,"r").read()

In [5]:
print(text[:700]) # as we can see above every 650-700 character we have a sonet


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When fo


In [0]:
vocab = sorted(set(text)) # we're gonna build our text generation with character based

In [7]:
vocab # therefore our vocabulary consist characters not words

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '}']

In [8]:
len(vocab) # Lenght of vocabulary

84

Processing


In [9]:
# we create dictionary for value for every character

for pair in enumerate(vocab):
  print(pair)

(0, '\n')
(1, ' ')
(2, '!')
(3, '"')
(4, '&')
(5, "'")
(6, '(')
(7, ')')
(8, ',')
(9, '-')
(10, '.')
(11, '0')
(12, '1')
(13, '2')
(14, '3')
(15, '4')
(16, '5')
(17, '6')
(18, '7')
(19, '8')
(20, '9')
(21, ':')
(22, ';')
(23, '<')
(24, '>')
(25, '?')
(26, 'A')
(27, 'B')
(28, 'C')
(29, 'D')
(30, 'E')
(31, 'F')
(32, 'G')
(33, 'H')
(34, 'I')
(35, 'J')
(36, 'K')
(37, 'L')
(38, 'M')
(39, 'N')
(40, 'O')
(41, 'P')
(42, 'Q')
(43, 'R')
(44, 'S')
(45, 'T')
(46, 'U')
(47, 'V')
(48, 'W')
(49, 'X')
(50, 'Y')
(51, 'Z')
(52, '[')
(53, ']')
(54, '_')
(55, '`')
(56, 'a')
(57, 'b')
(58, 'c')
(59, 'd')
(60, 'e')
(61, 'f')
(62, 'g')
(63, 'h')
(64, 'i')
(65, 'j')
(66, 'k')
(67, 'l')
(68, 'm')
(69, 'n')
(70, 'o')
(71, 'p')
(72, 'q')
(73, 'r')
(74, 's')
(75, 't')
(76, 'u')
(77, 'v')
(78, 'w')
(79, 'x')
(80, 'y')
(81, 'z')
(82, '|')
(83, '}')


In [0]:
# dictionary which give us index according to char
char_to_ind = {char:ind for ind,char in enumerate(vocab)}


In [0]:
# numpy array which give us char according to index
ind_to_char = np.array(vocab)

In [12]:
char_to_ind["a"]

56

In [13]:
ind_to_char[56]

'a'

In [0]:
# we transform our full text to indexes
encoded_text = np.array([char_to_ind[c] for c in text])

In [15]:
encoded_text.shape
# we have 5445609 characters. 

(5445609,)

In [0]:
sample = text[:500]

In [17]:
sample

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bu"

In [18]:
encoded_text[:500]

array([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 12,  0,  1,  1, 31, 73, 70, 68,  1, 61, 56, 64,
       73, 60, 74, 75,  1, 58, 73, 60, 56, 75, 76, 73, 60, 74,  1, 78, 60,
        1, 59, 60, 74, 64, 73, 60,  1, 64, 69, 58, 73, 60, 56, 74, 60,  8,
        0,  1,  1, 45, 63, 56, 75,  1, 75, 63, 60, 73, 60, 57, 80,  1, 57,
       60, 56, 76, 75, 80,  5, 74,  1, 73, 70, 74, 60,  1, 68, 64, 62, 63,
       75,  1, 69, 60, 77, 60, 73,  1, 59, 64, 60,  8,  0,  1,  1, 27, 76,
       75,  1, 56, 74,  1, 75, 63, 60,  1, 73, 64, 71, 60, 73,  1, 74, 63,
       70, 76, 67, 59,  1, 57, 80,  1, 75, 64, 68, 60,  1, 59, 60, 58, 60,
       56, 74, 60,  8,  0,  1,  1, 33, 64, 74,  1, 75, 60, 69, 59, 60, 73,
        1, 63, 60, 64, 73,  1, 68, 64, 62, 63, 75,  1, 57, 60, 56, 73,  1,
       63, 64, 74,  1, 68, 60, 68, 70, 73, 80, 21,  0,  1,  1, 27, 76, 75,
        1, 75, 63, 70, 76,  1, 58, 70, 69, 75, 73, 56, 58, 75, 60, 59,  1,
       75, 70,  1, 75, 63

In [0]:
seq_len = 120
# our sequence length for every sequence

In [0]:
total_number_seq = len(text) // (seq_len + 1)

In [21]:
total_number_seq
# we're gonna have 45005 sequences

45005

In [0]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
# we create char dataset from tensorflow Dataset using encoded text

In [0]:
sequences = char_dataset.batch(seq_len+1,drop_remainder=True)
# we're creating our sequencess with batch according to sequence length

In [0]:
def create_seq_targets(seq):
  input_txt = seq[:-1] # Okan i
  target_txt = seq[1:] # kan is
  return input_txt,target_txt

In [0]:
# mapping sequences to create input and target data
dataset = sequences.map(create_seq_targets)

In [26]:
for input_txt,target_txt in dataset.take(1):
  print(input_txt.numpy())
  print("".join(ind_to_char[input_txt.numpy()]))
  print("\n")
  print(target_txt.numpy())
  print("".join(ind_to_char[target_txt.numpy()]))
  # simple input and target data output

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [0]:
batch_size = 128
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [28]:
dataset
# we have batches one batch containes 128 sequence
# we have lenght of sequence : 120

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

In [0]:
# for embedding layer
embed_dim = 64
rnn_neurons = 1536

In [0]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [0]:
def create_model(vocab_size,embed_dim,rnn_neurons,batch_size):

  model = Sequential()

  model.add(Embedding(vocab_size,embed_dim,batch_input_shape=[batch_size,None]))
  # vocab_size -> input dimension
  # embed_dim -> output dimension
  # batch_input_size -> specify batch_size
  model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer="glorot_uniform"))
  # return_sequences = true -> we return last output sequence
  # stateful = true -> we keep current state
  model.add(Dense(vocab_size))
  # output layer

  model.compile("adam",loss=sparse_cat_loss)

  return model

In [0]:
vocab_size = len(vocab)
model = create_model(vocab_size,embed_dim,rnn_neurons,batch_size)
# we build our model

In [33]:
model.summary()
# model summary

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1536)         7382016   
_________________________________________________________________
dense (Dense)                (128, None, 84)           129108    
Total params: 7,516,500
Trainable params: 7,516,500
Non-trainable params: 0
_________________________________________________________________


In [0]:
epochs = 30

In [35]:
# we predict random batch before training
for input_example_batch, target_example_batch in dataset.take(1):

  example_batch_predictions = model(input_example_batch)

  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")

(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


In [36]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))


Given the input seq: 


    Fare you well, my dove!
  Laer. Hadst thou thy wits, and didst persuade revenge,
    It could not move thus.  
  Op


Next Char Predictions: 

mgZz0OcUX
X&;1CITEZ|uJtwiUWci6fGibvTSOF>]7"!`fSFOGYOlLl"xBhmf>yV"TI?7.KQSF)}o|7sxE!pM7I35hI`9"2}ncAsHX(iY6rxu'ClfQ"`5;w2


In [37]:
model.fit(dataset,epochs=epochs)

Epoch 1/30
351/351 [==============================] - 80s 227ms/step - loss: 2.4586
Epoch 2/30
351/351 [==============================] - 80s 229ms/step - loss: 1.6453
Epoch 3/30
351/351 [==============================] - 81s 230ms/step - loss: 1.4037
Epoch 4/30
351/351 [==============================] - 81s 230ms/step - loss: 1.2994
Epoch 5/30
351/351 [==============================] - 81s 230ms/step - loss: 1.2409
Epoch 6/30
351/351 [==============================] - 80s 229ms/step - loss: 1.1996
Epoch 7/30
351/351 [==============================] - 80s 229ms/step - loss: 1.1688
Epoch 8/30
351/351 [==============================] - 80s 228ms/step - loss: 1.1421
Epoch 9/30
351/351 [==============================] - 81s 230ms/step - loss: 1.1189
Epoch 10/30
351/351 [==============================] - 81s 231ms/step - loss: 1.0976
Epoch 11/30
351/351 [==============================] - 81s 231ms/step - loss: 1.0771
Epoch 12/30
351/351 [==============================] - 81s 231ms/step - lo

In [0]:
# In first model experience epoch was 40 losss was 1.23.
# So i find boundary loss increase which was 30. epoch and train new model with 30 epochs
#As we can see above after 28.th epoch model loss is increasing therefore we should stop here.

In [0]:
model.save('my_model.h5') 

In [0]:
created_model = create_model(vocab_size,embed_dim,rnn_neurons,batch_size=1)
 # we specify batch size because we give restricted data

In [0]:
created_model.load_weights("my_model.h5")
# we load our weights

In [0]:
created_model.build(tf.TensorShape([1,None]))
# we build our model

In [43]:
created_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1536)           7382016   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             129108    
Total params: 7,516,500
Trainable params: 7,516,500
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model,start_seed,gen_size=500,temp=1.0):
  
  num_generate = gen_size
  
  input_eval = [char_to_ind[s] for s in start_seed]
  # we vectorize our start seed

  input_eval = tf.expand_dims(input_eval,0)
  # expand dimensions to correct shape

  text_generated = []

  temperature = temp

  model.reset_states()
  # reset states

  # we iterate until the reach num_generate
  for i in range(num_generate):

    #we get prediction
    predictions = model(input_eval)

    # we use squeezing for undo expand dimensions
    predictions = tf.squeeze(predictions,0)

    # actual predictions based on temp
    predictions = predictions / temperature

    #we get id of our character
    predicted_id = tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()

    # we assign next character
    input_eval = tf.expand_dims([predicted_id],0)

    # we get character and append our output
    text_generated.append(ind_to_char[predicted_id])

  return (start_seed+"".join(text_generated))

In [50]:
print(generate_text(created_model,"ROMEO",gen_size=1000))
# example output - 1

ROMEO. Poor shows his bondard, he is now ay arms.
  APEMANTUS. Ther. O, what, what?
  AGUECHEEK. Is not that a calf?  
  CHARMIAN. Good naw, good company. I will be Killia'- I, or being RICHMOND'S house

Enter PISANIO, Antonio,
     bewitchance
    t   gentle lady, in mine against thy talk,
    The matter-
    Is busily and wash his purse?
    One of our mother, and of your end-
    But, soft and want, defiechet, alise do hang in't.  
  ISABELLA. It is Apalanth too, and fast and loathed witzon
     Die, thou shouldst adjoin to woo thee;
    Here comes the hot balm of the argument.
    Let me see.
  ANTIGONUS. If it please the Emperor,'s ears to view,
    Can we ne'er to distemper. Fear you so,
    Could men express me, and rest here with this,
    That e of her
    Shall is too short, a prize and Say quickly;
    Then farewell. Now sir, I have manklething worse;
    For, by thee, where your son small I ends me; straight.
ANTIPHOLUS OF MRS. OVERIONE. What says Jupiter
    What you have 

In [55]:
print(generate_text(created_model,"JULIET",gen_size=200))
# example output - 2

JULIET. Sau your daughter?
  PAULINA. What, Yea,!
    And, for my self-expeciatel was by-
    Her bridal-conceite can every wash
    All to myself the moon!          Exeunt

            Enter a WALES pash o
